In [9]:
%matplotlib inline
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
from PIL import Image
import shutil
import pandas as pd
import cv2
from IPython.display import clear_output
import os
from sklearn import preprocessing
import matplotlib.patches as patches
import time
from sklearn.decomposition import PCA
from termcolor import colored
from google.colab import output
import pickle

In [10]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/MyDrive')

Drive already mounted at /content/MyDrive; to attempt to forcibly remount, call drive.mount("/content/MyDrive", force_remount=True).


In [11]:
%mkdir -p /content/Eval_Data
!cp /content/MyDrive/MyDrive/0-MasterThesis/Eval_Data.zip /content/Eval_Data
%cd /content/Eval_Data/
!unzip Eval_Data.zip
%cd /content/

/content/Eval_Data
Archive:  Eval_Data.zip
replace Danilo_data/IR_information.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: /content


In [12]:
Haak1_true_df = pd.read_csv(r'/content/Eval_Data/Haak_data/Flight1/Haak1_true_annot.csv')
Haak1_anomalies_object_df = pd.read_csv(r'/content/Eval_Data/Haak_data/Flight1/Haak1_ano_object_annot.csv')
Haak1_anomalies_land_df = pd.read_csv(r'/content/Eval_Data//Haak_data/Flight1/Haak1_ano_land_annot.csv')

Haak3_true_df = pd.read_csv(r'/content/Eval_Data//Haak_data/Flight3/Haak3_true_annot.csv')
Haak3_anomalies_object_df = pd.read_csv(r'/content/Eval_Data//Haak_data/Flight3/Haak3_ano_object_annot.csv')
Haak3_anomalies_land_df = pd.read_csv(r'/content/Eval_Data//Haak_data/Flight3/Haak3_ano_land_annot.csv')

Danilo_true_df = pd.read_csv(r'/content/Eval_Data//Danilo_data/IR/Danilo_true_annot.csv')
Danilo_anomalies_land_df = pd.read_csv(r'/content/Eval_Data/Danilo_data/IR/Danilo_ano_land_annot.csv')
Empty_df = pd.read_csv(r'/content/Eval_Data/Danilo_data/IR/Empty.csv')

# Haak - Flight 1
print('Will you run it adaptive (y/n)? ')
adaptiveBool = input()

if adaptiveBool == 'y':
    isAdaptive = True

elif adaptiveBool == 'n':
    isAdaptive = False
else:
  raise Exception('Input error')

n = len(Haak1_true_df)
Haak1_true_train       = Haak1_true_df[:int(0.7*n)]
Haak1_true_test        = Haak1_true_df[int(0.7*n):]
Haak1_ano_object       = Haak1_anomalies_object_df
Haak1_ano_land         = Haak1_anomalies_land_df
 
# Haak - Flight 1
Haak3_true             = Haak3_true_df
Haak3_ano_object       = Haak3_anomalies_object_df
Haak3_ano_land         = Haak3_anomalies_land_df

# Danilo
Danilo_true            = Danilo_true_df
Danilo_ano_land        = Danilo_anomalies_land_df
empty                  = Empty_df


'''
Haak1_true_train | Haak1_true_test
Haak1_ano_object
Haak1_ano_land

Haak3_true 
Haak3_ano_object 
Haak3_ano_land 

Danilo_true
empty
Danilo_ano_land
'''

Will you run it adaptive (y/n)? 
y


'\nHaak1_true_train | Haak1_true_test\nHaak1_ano_object\nHaak1_ano_land\n\nHaak3_true \nHaak3_ano_object \nHaak3_ano_land \n\nDanilo_true\nempty\nDanilo_ano_land\n'

# EVALUATION

In [ ]:
##################################################### HAAK 1
directory = '/content/Eval_Data/Haak_data/Flight1/20170926_norm_1100-3900/' # 8-bit
scenery = 'Haak1'
object_df = Haak1_ano_object
land_df   = Haak1_ano_land
true_df   = Haak1_true_test
resizeInput  = False
fps = 7
Haak1 = True

##################################################### HAAK 3
directory = '/content/Eval_Data/Haak_data/Flight3/20170926_norm_1100-3900/' # 8-bit
scenery = 'Haak3'
object_df = Haak3_ano_object
land_df   = Haak3_ano_land
true_df       = Haak3_true
resizeInput  = False
fps = 7
Haak1 = False

##################################################### DANILO
directory = '/content/Eval_Data/Danilo_data/IR/All-Frames-Simple/' # 8-bit
scenery = 'Danilo'
object_df = empty
land_df   = Danilo_ano_land
true_df       = Danilo_true
resizeInput  = True
fps = 1
Haak1 = False

In [22]:
#####################################################
#MOG_train = cv2.createBackgroundSubtractorMOG2(history=10000, detectShadows=0, varThreshold = 16) # NON-ADAPTVE
history = 1
MOG_train = cv2.createBackgroundSubtractorMOG2(history=history, detectShadows=0, varThreshold = 16) # ADAPTIVE (500 default) (here: seconds)

#history = len(Haak1_true_train)
#MOG_train = cv2.createBackgroundSubtractorMOG2(history=history, detectShadows=0, varThreshold = 16) # ADAPTIVE (500 default) (here: seconds)

#history = 420
#MOG_train = cv2.bgsegm.createBackgroundSubtractorMOG(history=history, nmixtures=2)
#####################################################

# TRAIN

In [23]:
##################################################### HAAK 1
directory = '/content/Eval_Data/Haak_data/Flight1/20170926_norm_1100-3900/' # 8-bit
scenery = 'Haak1'
df = Haak1_true_train
resizeInput  = False
fps = 7

#####################################################
n_files = len(df)
start = 0
end = n_files

if isAdaptive:
  end = history

simplePlot = False

#####################################################
j = 0
for i, row in df[start:end].iterrows():
    j += 1
    ############################
    output.clear()
    print(str(j) + '/' + str(n_files))
    file = row['file']
    
    img = np.array(Image.open(os.path.join(directory, file)))
    #img = cv2.imread(os.path.join(directory, file))

    if resizeInput:
      img = resizeInput(img)

    ##################### ALGORITHM #####################
    start_time = time.time()
    mask = MOG_train.apply(img)
    print("--- %s seconds ---" % (time.time() - start_time))


    ##################### PLT TRAINING #################
    if simplePlot:
        clear_output(wait=True)
        fig = plt.figure(figsize=(15,10))

        plt.subplot(1,2,1)
        plt.imshow(img, cmap='gray', vmin=0, vmax=255)
        plt.colorbar()
        
        plt.subplot(1,2,2)
        plt.imshow(mask, cmap='gray', vmin=0, vmax=255)
        plt.colorbar()

        plt.show()

        print(str(j) + '/' + str(n_files))
        print(file)

1/12176
--- 0.006747007369995117 seconds ---


# Save Model

In [24]:
print(MOG_train.getNMixtures())
print(MOG_train.getHistory())
print(MOG_train.getBackgroundRatio())
#print(MOG_train.getBackgroundImage())
#print(MOG_train.getNoiseSigma())

5
1
0.8999999761581421


# TEST

In [ ]:
print('Is it adaptive? {}'.format(adaptiveBool))

Is it adaptive? n


In [15]:
##################################################### HAAK 3
directory = '/content/Eval_Data/Haak_data/Flight3/20170926_norm_1100-3900/' # 8-bit
scenery = 'Haak3'
object_df = Haak3_ano_object
land_df   = Haak3_ano_land
true_df       = Haak3_true
resizeInput  = False
fps = 7
Haak1 = False

#####################################################
area_threshold = 20
savePlot = False
UpdateWithoutLand = False
doPlots = False
#doExhaustivePlots = False # DOESNT DIFFER FROM NORMAL PLOT
doErrorValidation = False
doNMSValidation = False
#####################################################
files = sorted(os.listdir(directory))
n_files = len(files)
start = 0
end = n_files
#end = 50


object_metrics = {'FPs':0, 'FNs':0, 'TPs': 0}
land_metrics   = {'FPs':0, 'FNs':0, 'TPs': 0}
true_metrics   = {'FPs':0, 'FNs':0, 'TPs': 0}
j = start
#####################################################
for i, file in enumerate(files[start:end]):
    #time.sleep(0.5)
    j += 1
    ############################
    true_boxes = []
    pred_boxes = []
    intersection_boxes = []
    FP, FN, TP = 0,0,0
    ############################
    #output.clear()
    print(str(j) + '/' + str(n_files))

    isAnomObject = object_df['file'].str.contains(file).sum()
    isAnomLand   = land_df['file'].str.contains(file).sum()
    isTrue       = true_df['file'].str.contains(file).sum()
    
    isTrueTrain = True if (Haak1_true_train['file'].str.contains(file).sum() and Haak1) else False
    
    if (not isAdaptive) and isTrueTrain:
      print('Skipping')
      continue

    
    img = np.array(Image.open(os.path.join(directory, file)))
    #img = cv2.imread(os.path.join(directory, file))

    if resizeInput:
      img = resizeInputFunc(img)

    # DONT WANT ADAPTIVE TO APlpY TO EDGE CASES FRAMES
    if not (isAnomObject or isAnomLand or isTrue or isTrueTrain):
      continue


    ##################### ALGORITHM #####################
    start_time = time.time()

    #img = cv2.GaussianBlur(img.copy(),(5, 5), 0)

    if isAdaptive:
      if isTrueTrain:
        #if (j%fps == 0):
        #  print('TRAIN DATA - UPDATE')
        #  _ = MOG_train.apply(img)
        #print('TRAIN DATA - NOTTT UPDATE')

        _ = MOG_train.apply(img)
        continue

      # Special case if not update on land (but still evaluate)
      if isAnomLand and UpdateWithoutLand:
        MOG_test = MOG_train
        mask = MOG_test.apply(img)

      # Special case update at 1 fps
      #elif j%fps != 0:
      #  print('NOT UPDATE')
      #  MOG_test = MOG_train
      #  mask = MOG_test.apply(img)

      else:
        mask = MOG_train.apply(img)

      # Dont want to evaluate on traning data, but want to update model

    else:
      MOG_test = MOG_train
      mask = MOG_test.apply(img)

    print("--- %s seconds ---" % (time.time() - start_time))
    #####################################################

    if isAnomObject:
        scenery = 'Object'
        true_boxes = object_df[object_df['file'] == file]
        true_boxes = true_boxes[['x','y','w','h']].to_numpy()      
    elif isAnomLand:
        scenery = 'Land'
        true_boxes = land_df[land_df['file'] == file]
        true_boxes = true_boxes[['x','y','w','h']].to_numpy()    
    elif isTrue:
        scenery = 'Nothing'
    else:
        print('file {} is not assessed (edge case)?'.format(file))
        continue

    # Pred Boxes
    pred_boxes = boxes_from_mask(mask, area_threshold)
    
    if (not isAnomLand) and (len(pred_boxes) >= 2):
        if doNMSValidation: 
            plot_results(img, mask, pred_boxes, true_boxes, intersection_boxes, 'N/A')
            print(colored('NON MAXIMAL SUPRESSION', 'magenta'))
            print('Prior: {}'.format(pred_boxes))
            pred_boxes = pred_box_IoU_cancellation(pred_boxes)
            print('Post: {}'.format(pred_boxes))
            _ = input()
        else:
            pred_boxes = pred_box_IoU_cancellation(pred_boxes)
    
    pre_land_boxes = []
    if isAnomLand:
        pre_land_boxes = pred_boxes
        pred_boxes = covering_rectangle_from_square_boxes(pred_boxes) # Combine small boxes

    # Calculate accuracy
    intersection_boxes, FP, FN, TP = compare_predictions_with_truth(pred_boxes, true_boxes)


    # Metrics
    if isAnomObject:
        update = {'FPs':FP, 'FNs':FN, 'TPs': TP}
        object_metrics = {i: object_metrics.get(i, 0) + update.get(i, 0) for i in set(update)}
    elif isAnomLand:
        update = {'FPs':FP, 'FNs':FN, 'TPs': TP}
        land_metrics = {i: land_metrics.get(i, 0) + update.get(i, 0) for i in set(update)}
    elif isTrue:
        update = {'FPs':FP, 'FNs':FN, 'TPs': TP}
        true_metrics = {i: true_metrics.get(i, 0) + update.get(i, 0) for i in set(update)}

    # Plot
    title = 'FP: {}, FN: {}, TP: {} | {}'.format(FP, FN, TP, scenery)

    if doPlots:
        plot_results(img, mask, pred_boxes, true_boxes, intersection_boxes, title)
        
    #if doExhaustivePlots:
    #    exhaustive_plot_results(img, img_canny, img_dil, mask, pred_boxes, true_boxes, intersection_boxes, pre_land_boxes, title)

    if doErrorValidation:
        if FP:
            plot_results(img, mask, pred_boxes, true_boxes, intersection_boxes, title)
            print(colored('FALSE POSITIVE, total: {}'.format(object_metrics['FPs']+land_metrics['FPs']+true_metrics['FPs']), 'cyan'))
            print(row)
            print(pred_boxes)
            _ = input()
        if FN:
            plot_results(img, mask, pred_boxes, true_boxes, intersection_boxes, title)
            print(colored('FALSE NEGATIVE, total: {}'.format(object_metrics['FNs']+land_metrics['FNs']+true_metrics['FNs']), 'red'))
            print(row)
            _ = input()




    # DONT WANT TO SAVE TEHSE CASES
    if not (isAnomObject or isAnomLand or isTrue):
      #print('Skipping')
      continue
    if savePlot:
        
        im_mask = Image.fromarray(mask)
        #method = 'MixtureModelMOG2_Adaptive_60sec'
        #method = 'MixtureModelMOG2_Adaptive_60sec'
        method = 'MixtureModelMOG2_NONAdaptive'
        flight = 'Haak1'
        im_mask.save(os.path.join('/content/MyDrive/MyDrive/0-MasterThesis/Methods/', str(method), str(flight), '{0:05d}'.format(i) + '.png'))



#        Flight 1    Flight 2      Flight 3
#   Rec
#   Prec

print('Len true: {}, len obj: {}, len land: {}'.format(len(true_df), len(object_df), len(land_df)))
print('Object: {}'.format(object_metrics))
print('Recall: {:.4f}'.format(object_metrics['TPs']/(object_metrics['FNs']+object_metrics['TPs']+0.00000001)))
print('Precision: {:.4f}'.format(object_metrics['TPs']/(object_metrics['FPs']+object_metrics['TPs']+0.00000001)))
print('--------')
print('Land: {}'.format(land_metrics))
print('Recall: {:.4f}'.format(land_metrics['TPs']/(land_metrics['FNs']+land_metrics['TPs']+0.00000001)))
print('Precision: {:.4f}'.format(land_metrics['TPs']/(land_metrics['FPs']+land_metrics['TPs']+0.00000001)))
print('--------')
print('True: {}'.format(true_metrics))
print('Recall: {:.4f}'.format(true_metrics['TPs']/(true_metrics['FNs']+true_metrics['TPs']+0.00000001)))
print('Precision: {:.4f}'.format(true_metrics['TPs']/(true_metrics['FPs']+true_metrics['TPs']+0.00000001)))

1/6893
--- 0.019733905792236328 seconds ---
2/6893
--- 0.02737879753112793 seconds ---
3/6893
--- 0.023668289184570312 seconds ---
4/6893
--- 0.021208524703979492 seconds ---
5/6893
--- 0.020975589752197266 seconds ---
6/6893
--- 0.014636039733886719 seconds ---
7/6893
8/6893
--- 0.008068084716796875 seconds ---
9/6893
--- 0.007507801055908203 seconds ---
10/6893
--- 0.018706321716308594 seconds ---
11/6893
--- 0.007566690444946289 seconds ---
12/6893
--- 0.007915496826171875 seconds ---
13/6893
--- 0.008353233337402344 seconds ---
14/6893
--- 0.007659912109375 seconds ---
15/6893
--- 0.028597116470336914 seconds ---
16/6893
--- 0.031186580657958984 seconds ---
17/6893
--- 0.017652511596679688 seconds ---
18/6893
--- 0.014974355697631836 seconds ---
19/6893
--- 0.007611513137817383 seconds ---
20/6893
--- 0.014797449111938477 seconds ---
21/6893
--- 0.02592182159423828 seconds ---
22/6893
--- 0.007508516311645508 seconds ---
23/6893
--- 0.022876501083374023 seconds ---
24/6893
--- 0.03

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:123: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Streaming output truncated to the last 5000 lines.
--- 0.005002021789550781 seconds ---
4351/6893
--- 0.00409388542175293 seconds ---
4352/6893
--- 0.004055023193359375 seconds ---
4353/6893
--- 0.004064083099365234 seconds ---
4354/6893
--- 0.004048824310302734 seconds ---
4355/6893
--- 0.004101991653442383 seconds ---
4356/6893
--- 0.0043604373931884766 seconds ---
4357/6893
--- 0.0040814876556396484 seconds ---
4358/6893
--- 0.005079984664916992 seconds ---
4359/6893
--- 0.004101991653442383 seconds ---
4360/6893
--- 0.006272792816162109 seconds ---
4361/6893
--- 0.00404047966003418 seconds ---
4362/6893
--- 0.004324436187744141 seconds ---
4363/6893
--- 0.004061222076416016 seconds ---
4364/6893
4365/6893
--- 0.004305839538574219 seconds ---
4366/6893
--- 0.004148721694946289 seconds ---
4367/6893
--- 0.004121303558349609 seconds ---
4368/6893
--- 0.004086494445800781 seconds ---
4369/6893
--- 0.005401134490966797 seconds ---
4370/6893
--- 0.004061698913574219 seconds ---
4371/6893

In [ ]:
# PLACEHOLDER

In [7]:
def resizeInputFunc(img):

    img_resized = cv2.resize(img, (640,512), interpolation = cv2.INTER_CUBIC)

    return img_resized

def pred_box_IoU_cancellation(pred_boxes):
    
    boxes_to_save = np.ones(len(pred_boxes))
    for i, box1 in enumerate(pred_boxes):       
        for j, box2 in enumerate(pred_boxes):
            if i == j:
                continue

            _, _, _, _, A_intsc, A_union, IoU, A1, A2  = intersection_info(box1, box2)
            
            # if two predicted boxes are almost equal (50% of area in another, remove the smallest)
            if (A_intsc > 0.25*A1) or (A_intsc > 0.25*A2):
                if A1 >= A2:
                    boxes_to_save[j] = 0
                else:
                    boxes_to_save[i] = 0
                  
    boxes = []
    for k, boolean in enumerate(boxes_to_save):
        if boolean:
            box = pred_boxes[k]
            boxes = [*boxes, box]
    boxes = np.array(boxes)
    
    return boxes


def covering_rectangle_from_square_boxes(boxes):
    output = np.array([])
    
    if len(boxes):
        x_left = np.min(boxes[:,0])
        width = np.max(boxes[:,0] + boxes[:,2] - x_left)
        y_top = np.min(boxes[:,1])
        height = np.max(boxes[:,1] + boxes[:,3] - y_top)
        
        output = np.array([[x_left, y_top, width, height]])

    return output

def intersection_info(box_i, box_j):
    # NMS
    x_left_i, y_upper_i, w_i, h_i = box_i
    x_right_i = x_left_i + w_i
    y_lower_i = y_upper_i + h_i
    area_i = w_i*h_i

    x_left_j, y_upper_j, w_j, h_j = box_j
    x_right_j = x_left_j + w_j
    y_lower_j = y_upper_j + h_j
    area_j = w_j*h_j

    x_left  = max(x_left_i, x_left_j)
    y_upper = max(y_upper_i, y_upper_j)

    x_right = min(x_right_i, x_right_j)
    y_lower = min(y_lower_i, y_lower_j)

    w_intersec = max(0, x_right - x_left)
    h_intersec = max(0, y_lower - y_upper)

    intersection_area = w_intersec*h_intersec
    union_area = area_i + area_j - intersection_area
    IoU = intersection_area / union_area
    
    return list([x_left, y_upper, w_intersec, h_intersec, intersection_area, union_area, IoU, area_i, area_j])

def compare_predictions_with_truth(pred_boxes, true_boxes):
    
    false_positives = 0
    false_negatives = 0
    true_positives = 0
    
    
    intersection_boxes = []
    match_found_pred = np.ones(len(pred_boxes))
    match_found_true = np.ones(len(true_boxes))
    
    for i, box_pred in enumerate(pred_boxes):
        for j, box_true in enumerate(true_boxes):
                     
            x_left, y_upper, w_intsc, h_intsc, A_intsc, A_union, IoU, A_pred, A_true  = intersection_info(box_pred, box_true)

            # If IoU big enough
            if IoU >= 0.1:
                match_found_pred[i] = 0
                match_found_true[j] = 0
                intersection_boxes = [*intersection_boxes, [x_left, y_upper, w_intsc, h_intsc]]
            
            # If within on another, and area large enough
            elif (IoU >= 0.05) and ((A_intsc>=A_pred) or (A_intsc>=A_true)):
                match_found_pred[i] = 0
                match_found_true[j] = 0
                intersection_boxes = [*intersection_boxes, [x_left, y_upper, w_intsc, h_intsc]]
                
            # If within on another, but area small, only false neg, not false positve
            elif (IoU < 0.05) and ((A_intsc>=A_pred) or (A_intsc>=A_true)):
                match_found_pred[i] = 0
            
    intersection_boxes = np.array(intersection_boxes)
        
    false_positives = match_found_pred.sum()
    false_negatives = match_found_true.sum()
    true_positives  = len(match_found_true) - false_negatives
                
    return intersection_boxes, false_positives, false_negatives, true_positives

def boxes_from_mask(mask, area_threshold):
    
    contours, hierarchy = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, 2)
    
    # Threshold by area
    contours_areathrsh = []
    for cnt in contours:
        if cv2.contourArea(cnt) > area_threshold:
            contours_areathrsh = [*contours_areathrsh, cnt]
    contours_areathrsh = np.array(contours_areathrsh)
    
    # combine boxes thresholded
    boxes = []
    for i, cnt in enumerate(contours_areathrsh):
        box = cv2.boundingRect(cnt)
        boxes = [*boxes, box]
    boxes = np.array(boxes)

    return boxes

def plot_results(img, mask, pred_boxes, true_boxes, intersection_boxes, title):
    colors = ['r', 'c', 'g', 'b', 'y', 'm']
    
    clear_output(wait=True)
    fig = plt.figure(figsize=(21,10))

    # Plot boxes - Area thresholded
    plt.subplot(1,2,1)
    plt.imshow(img, cmap='gray', vmin=0, vmax=255)
    ax = plt.gca()
    
    plt.subplot(1,2,2)
    plt.imshow(mask, cmap='gray', vmin=0, vmax=255)
    ax = plt.gca()

    for i, box in enumerate(intersection_boxes):
        x,y,w,h = box # upper left
        rect = patches.Rectangle((x,y), w,h, edgecolor='y', facecolor='y', fill=True, linewidth=2, alpha=0.25)
        ax.add_patch(rect)     
    
    for i, box in enumerate(pred_boxes):
        x,y,w,h = box # upper left
        rect = patches.Rectangle((x,y), w,h, edgecolor='r', fill=False, linewidth=2)
        ax.add_patch(rect)
        
    for i, box in enumerate(true_boxes):
        x,y,w,h = box # upper left
        rect = patches.Rectangle((x,y), w,h, edgecolor='g', fill=False, linewidth=2)
        ax.add_patch(rect)           
 
    fig.suptitle(title + ' | Found: {}, Truth: {}'.format(len(pred_boxes), len(true_boxes)), fontsize=18)
    
    plt.show()
    
    
def exhaustive_plot_results(img, img_canny, img_dil, mask, pred_boxes, true_boxes, intersection_boxes, pre_land_boxes, title):
    colors = ['r', 'c', 'g', 'b', 'y', 'm']
    
    clear_output(wait=True)
    fig = plt.figure(figsize=(21,28))

    # Plot boxes - Area thresholded
    plt.subplot(3,2,1)
    plt.imshow(img, cmap='gray', vmin=0, vmax=255)
    
    plt.subplot(3,2,2)
    plt.imshow(img_canny, cmap='gray')
    plt.colorbar()
    
    plt.subplot(3,2,3)
    plt.imshow(img_dil, cmap='gray', vmin=0, vmax=255)
    
    plt.subplot(3,2,4)
    plt.imshow(mask, cmap='gray', vmin=0, vmax=255)
    ax = plt.gca()
    
    # Boxes in evaluation
    plt.subplot(3,2,5)
    plt.imshow(mask, cmap='gray', vmin=0, vmax=255)
    ax = plt.gca()

    for i, box in enumerate(intersection_boxes):
        x,y,w,h = box # upper left
        rect = patches.Rectangle((x,y), w,h, edgecolor='y', facecolor='y', fill=True, linewidth=2, alpha=0.25)
        ax.add_patch(rect)     
    
    for i, box in enumerate(pred_boxes):
        x,y,w,h = box # upper left
        rect = patches.Rectangle((x,y), w,h, edgecolor='r', fill=False, linewidth=4)
        ax.add_patch(rect)
        
    for i, box in enumerate(true_boxes):
        x,y,w,h = box # upper left
        rect = patches.Rectangle((x,y), w,h, edgecolor='g', fill=False, linewidth=4)
        ax.add_patch(rect)
        
    
    # All boxes
    plt.subplot(3,2,6)
    plt.imshow(mask, cmap='gray', vmin=0, vmax=255)
    ax = plt.gca()
    
    for i, box in enumerate(pred_boxes):
        x,y,w,h = box # upper left
        rect = patches.Rectangle((x,y), w,h, edgecolor='r', fill=False, linewidth=4)
        ax.add_patch(rect)
    
    for i, box in enumerate(pre_land_boxes):
        x,y,w,h = box # upper left
        rect = patches.Rectangle((x,y), w,h, edgecolor='r', fill=False, linewidth=4)
        ax.add_patch(rect)
        
    for i, box in enumerate(true_boxes):
        x,y,w,h = box # upper left
        rect = patches.Rectangle((x,y), w,h, edgecolor='g', fill=False, linewidth=4)
        ax.add_patch(rect)
    
    
    fig.suptitle(title + ' | Found: {}, Truth: {}'.format(len(pred_boxes), len(true_boxes)), fontsize=18)
    plt.show()

# PLOT PCA

In [ ]:
df = Haak1_true_train
directory = '/content/Eval_Data/Haak_data/Flight1/20170926_norm_1100-3900/' # 8-bit
n_files = int(12176/10)

min_all = 0
max_all = 255

imgs = []

# F1
for i, file in enumerate(df['file'][:n_files]):
    print(str(i) + '/' + str(n_files))
    #file = str(file[1]).split('.')[0][:] + '.png'

    img = np.array(Image.open(os.path.join(directory, file)))
    imgs = [*imgs, img.flatten()]

imgs = np.array(imgs)


'''
hist, edges = np.histogram(img.flatten(), 
                density=True, 
                bins=255,
                range=(0,255) )

histDAN = [*histDAN, hist]

histDAN = np.array(histDAN)
edgesDAN = np.array(edges)
'''

In [ ]:
from sklearn.decomposition import PCA
import time
%matplotlib inline

start_time = time.time()

X = imgs
X_PCA = PCA(n_components = 25).fit(X)

print("--- %s seconds ---" % (time.time() - start_time))

#components = 25
#X = train_imgs_flat
#X_PCA = PCA(n_components = 3).fit(X)

In [ ]:
#########
# F1
import matplotlib.pyplot as plt
import numpy as np
import datetime
import matplotlib.dates as mdates
from matplotlib.transforms import Transform
import matplotlib.patheffects as pe
import numpy as np
from scipy.integrate import simps
from numpy import trapz
from matplotlib.ticker import (
    AutoLocator, AutoMinorLocator)
plt.figure().clear()
plt.close()
plt.cla()
plt.clf()

fig, ax = plt.subplots(figsize=(15, 3))
fontsize = 24
from matplotlib import rc
rc('font',**{'family':'serif','serif':['Palatino']})
plt.rcParams['pdf.fonttype'] = 42
plt.grid(True)

plt.bar(np.arange(1,components+1),X_PCA.explained_variance_ratio_[:components])
plt.plot(np.arange(1,components+1), np.cumsum(X_PCA.explained_variance_ratio_[:components]),
         color="darkorange", marker="D")


plt.xlim((0,9))
#plt.ylim((0.001,0.155))
plt.xlabel('Principal components', fontsize=fontsize, fontname='Palatino')
plt.ylabel('Explained variance', fontsize=fontsize, fontname='Palatino')
plt.xticks(fontsize=fontsize-3, fontname='Palatino')
plt.yticks(fontsize=fontsize-3, fontname='Palatino')

#ax.set_yscale('log')
frame1 = plt.gca()
#frame1.axes.xaxis.set_ticklabels([])
#frame1.axes.yaxis.set_ticklabels([])

plt.legend(fontsize=fontsize, prop={'size':fontsize-4, 'family': 'Palatino'})
#plt.title('Histogram of IQ')
#plt.text(60, .025, r'$\mu=100,\ \sigma=15$')
#plt.xlim(40, 160)
#plt.ylim(0, 0.03)

plt.savefig('/Users/ola/Documents/Masteroppgave/Figures/PCA_pareto.pdf', bbox_inches='tight')

plt.show()

(12176,)